In [1]:
import warnings
warnings.filterwarnings("ignore")
import joblib
import gc
import multiprocessing as mp
import numpy as np
import pandas as pd
import _pickle as cpickle
import pickle
from datetime import datetime
import time
from pandas import HDFStore
import lightgbm as lgb
from lightgbm.sklearn import LGBMClassifier
import xgboost
from xgboost.sklearn import XGBClassifier
import xgboost as xgb
from operator import itemgetter
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import check_cv, train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, roc_auc_score, log_loss, auc

In [2]:
train = pd.read_pickle("data/final_train.pkl")
x_train = train.drop(['order_id'], axis=1)
del train;
gc.collect()
test = pd.read_pickle("data/final_test.pkl")
x_test = test.drop(["order_id"], axis=1)
labels = pd.read_pickle("data/final_labels.pkl")

In [3]:
x_train.shape, x_test.shape

((8474661, 79), (4833292, 78))

In [4]:
class CustomStackingClassifier:
    def __init__(self, estimators, random_state, params, nround, 
                 version, loop=3,
                 valid_size=0.05, stratify=True, verbose=1,
                 early_stopping=60, use_probas=True):
        self.clf = estimators
        self.mod=cpickle
        self.loop = loop
        self.params = params
        self.nround = nround    
        self.version = version
        self.valid_size = valid_size
        self.verbose = verbose
        self.random_state = random_state
        self.early_stopping = early_stopping
        self.models = []


    def split_build_valid(self, train_user, X_train, y_train):
        train_user['is_valid'] = np.random.choice(
            [0,1],
            size=len(train_user),
            p=[1-self.valid_size, self.valid_size])

        valid_n = train_user['is_valid'].sum()
        build_n = (train_user.shape[0] - valid_n)
        
        print('build user:{}, valid user:{}'.format(build_n, valid_n))
        valid_user = train_user[train_user['is_valid']==1].user_id
        is_valid = X_train.user_id.isin(valid_user)
        
        dbuild = lgb.Dataset(X_train[~is_valid].drop('user_id', axis=1),
                             y_train[~is_valid],
                             categorical_feature=['product_id', 'aisle_id', 'department_id'])
        dvalid = lgb.Dataset(X_train[is_valid].drop('user_id', axis=1),
                             label=y_train[is_valid],
                             categorical_feature=['product_id', 'aisle_id', 'department_id'])
        watchlist_set = [dbuild, dvalid]
        watchlist_name = ['build', 'valid']
        
        print('FINAL SHAPE')
        print('dbuild.shape:{}  dvalid.shape:{}\n'.format(
            dbuild.data.shape,
            dvalid.data.shape))
        return dbuild, dvalid, watchlist_set, watchlist_name

    def fit(self, x, y):
        np.random.seed(self.random_state)
        train_user = x[['user_id']].drop_duplicates()

        for i in range(self.loop):
            dbuild, dvalid, watchlist_set, watchlist_name = self.split_build_valid(train_user, x, y)
            gc.collect();

            # Train models
            model = lgb.train(
                self.params,
                dbuild,
                self.nround,
                watchlist_set,
                watchlist_name,
                early_stopping_rounds=self.early_stopping,
                categorical_feature=['product_id', 'aisle_id', 'department_id'],
                verbose_eval=5)
            joblib.dump(model, "lgb_models/lgb_trained_{}_{}".format(self.version, i))
            self.models.append(model)
            del [dbuild, dvalid, watchlist_set, watchlist_name];
            gc.collect();
        del train_user;
        gc.collect()
        return self


    def predict(self, x, test_data):
#         dtest  = lgb.Dataset(x)
        sub_test = test_data[['order_id', 'product_id']]
        sub_test['yhat'] = 0
        for model in self.models:
            sub_test['yhat'] += model.predict(x)
        sub_test['yhat'] /= self.loop
        return sub_test

In [5]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss', 'auc'},
    'num_leaves': 256,
    'min_sum_hessian_in_leaf':20,
    'max_depth': 12,
    'learning_rate': 0.05,
    'feature_fraction': 0.6,
    # 'bagging_fraction': 0.9,
    # 'bagging_freq': 3,
    'verbose': 1
}

cscf_1 = CustomStackingClassifier(lgb, 71, params, 10000, 1)
cscf_2 = CustomStackingClassifier(lgb, 72, params, 10000, 2)
cscf_3 = CustomStackingClassifier(lgb, 73, params, 10000, 3)

In [6]:
cscf_1.fit(x_train, labels)
stack1 = cscf_1.predict(x_test, test)
stack1.to_csv("data/lgb_stack1.csv", index=False)

build user:124543, valid user:6666
FINAL SHAPE
dbuild.shape:(8044708, 78)  dvalid.shape:(429953, 78)

[LightGBM] [Info] Number of positive: 787000, number of negative: 7257708
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 2.873952 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 28196
[LightGBM] [Info] Number of data points in the train set: 8044708, number of used features: 78
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.097828 -> initscore=-2.221591
[LightGBM] [Info] Start training from score -2.221591
Training until validation scores don't improve for 60 rounds
[5]	build's auc: 0.832155	build's binary_logloss: 0.288275	valid's auc: 0.828539	valid's binary_logloss: 0.288105
[10]	build's auc: 0.833872	build's binary_logloss: 0.27253	valid's auc: 0.829417	valid's binary_logloss: 0.272974
[15]	build's auc: 0.835195	build's binary_logloss: 0.262692	valid's auc: 0.830561	valid's binary_logloss: 

[180]	build's auc: 0.852833	build's binary_logloss: 0.234994	valid's auc: 0.836539	valid's binary_logloss: 0.243113
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[185]	build's auc: 0.853011	build's binary_logloss: 0.234881	valid's auc: 0.836586	valid's binary_logloss: 0.243085
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[190]	build's auc: 0.853135	build's binary_logloss: 0.234807	valid's auc: 0.836599	valid's binary_logloss: 0.243078
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[195]	build's auc: 0.853287	build's binary_logloss: 0.234702	valid's auc: 0.836614	valid's binary_logloss: 0.243066
[LightGBM] 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[300]	build's auc: 0.855418	build's binary_logloss: 0.23341	valid's auc: 0.836623	valid's binary_logloss: 0.243036
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[305]	build's auc: 0.855534	build's binary_logloss: 0.23334	valid's auc: 0.836638	valid's binary_logloss: 0.24303
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[310]	build's auc: 0.

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[145]	build's auc: 0.851517	build's binary_logloss: 0.235952	valid's auc: 0.836768	valid's binary_logloss: 0.24002
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[150]	build's auc: 0.851672	build's binary_logloss: 0.235857	valid's auc: 0.836781	valid's binary_logloss: 0.240012
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning

[260]	build's auc: 0.854472	build's binary_logloss: 0.234088	valid's auc: 0.837171	valid's binary_logloss: 0.239777
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[265]	build's auc: 0.854637	build's binary_logloss: 0.233985	valid's auc: 0.837216	valid's binary_logloss: 0.239753
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[270]	build's auc: 0.854707	build's binary_logloss: 0.233943	valid's auc: 0.837222	valid's binary_logloss: 0.239749
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive

[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.097855 -> initscore=-2.221293
[LightGBM] [Info] Start training from score -2.221293
Training until validation scores don't improve for 60 rounds
[5]	build's auc: 0.832222	build's binary_logloss: 0.28834	valid's auc: 0.826337	valid's binary_logloss: 0.286949
[10]	build's auc: 0.833999	build's binary_logloss: 0.272572	valid's auc: 0.827232	valid's binary_logloss: 0.271941
[15]	build's auc: 0.835391	build's binary_logloss: 0.262715	valid's auc: 0.828495	valid's binary_logloss: 0.262635
[20]	build's auc: 0.836827	build's binary_logloss: 0.256124	valid's auc: 0.829378	valid's binary_logloss: 0.256613
[25]	build's auc: 0.838721	build's binary_logloss: 0.251446	valid's auc: 0.830054	valid's binary_logloss: 0.252596
[30]	build's auc: 0.839358	build's binary_logloss: 0.248424	valid's auc: 0.83062	valid's binary_logloss: 0.249912
[35]	build's auc: 0.840492	build's binary_logloss: 0.246109	valid's auc: 0.831133	valid's binary_logloss: 0.24804
[40]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[195]	build's auc: 0.853338	build's binary_logloss: 0.234737	valid's auc: 0.834827	valid's binary_logloss: 0.242756
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[200]	build's auc: 0.853418	build's binary_logloss: 0.234686	valid's auc: 0.83483	valid's binary_logloss: 0.242759
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[205]	build's auc: 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[310]	build's auc: 0.855711	build's binary_logloss: 0.233283	valid's auc: 0.835024	valid's binary_logloss: 0.242655
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[315]	build's auc: 0.855757	build's binary_logloss: 0.233256	valid's auc: 0.83501	valid's binary_logloss: 0.242662
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[420]	build's auc: 0.857734	build's binary_logloss: 0.232077	valid's auc: 0.835057	valid's binary_logloss: 0.242631
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[425]	build's auc: 0.857792	build's binary_logloss: 0.232043	valid's auc: 0.835057	valid's binary_logloss: 0.24263
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning

In [7]:
cscf_2.fit(x_train, labels)
stack1 = cscf_2.predict(x_test, test)
stack1.to_csv("data/lgb_stack2.csv", index=False)

build user:124537, valid user:6672
FINAL SHAPE
dbuild.shape:(8045207, 78)  dvalid.shape:(429454, 78)

[LightGBM] [Info] Number of positive: 786258, number of negative: 7258949
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 2.746533 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 28101
[LightGBM] [Info] Number of data points in the train set: 8045207, number of used features: 78
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.097730 -> initscore=-2.222705
[LightGBM] [Info] Start training from score -2.222705
Training until validation scores don't improve for 60 rounds
[5]	build's auc: 0.831897	build's binary_logloss: 0.288141	valid's auc: 0.83094	valid's binary_logloss: 0.291103
[10]	build's auc: 0.833682	build's binary_logloss: 0.272414	valid's auc: 0.831621	valid's binary_logloss: 0.275492
[15]	build's auc: 0.835056	build's binary_logloss: 0.262588	valid's auc: 0.832834	valid's binary_logloss: 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[185]	build's auc: 0.85289	build's binary_logloss: 0.234794	valid's auc: 0.839393	valid's binary_logloss: 0.244142
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[190]	build's auc: 0.853008	build's binary_logloss: 0.234721	valid's auc: 0.839407	valid's binary_logloss: 0.244135
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[195]	build's auc: 0.85314	build's binary_logloss: 0.234628	valid's auc: 0.83943	valid's binary_logloss: 0.24412
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[300]	build's auc: 0.855228	build's binary_logloss: 0.233358	valid's auc: 0.839651	valid's binary_logloss: 0.243977
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[305]	build's auc: 0.855335	build's binary_logloss: 0.233287	valid's auc: 0.839675	valid's binary_logloss: 0.243963
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[310]	build's auc: 0.855444	build's binary_logloss: 0.233226	valid's auc: 0.839685	valid's bi

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[410]	build's auc: 0.857289	build's binary_logloss: 0.232147	valid's auc: 0.839729	valid's binary_logloss: 0.243919
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[415]	build's auc: 0.857359	build's binary_logloss: 0.232106	valid's auc: 0.839731	valid's binary_logloss: 0.243916
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[420]	build's auc: 0.857443	build's binary_logloss: 0.232054	valid's auc: 0.83974	valid's bin

[50]	build's auc: 0.844006	build's binary_logloss: 0.241422	valid's auc: 0.833216	valid's binary_logloss: 0.250345
[55]	build's auc: 0.844569	build's binary_logloss: 0.240657	valid's auc: 0.83347	valid's binary_logloss: 0.249831
[60]	build's auc: 0.845385	build's binary_logloss: 0.239939	valid's auc: 0.833714	valid's binary_logloss: 0.249455
[65]	build's auc: 0.846114	build's binary_logloss: 0.239339	valid's auc: 0.833954	valid's binary_logloss: 0.249158
[70]	build's auc: 0.846916	build's binary_logloss: 0.238742	valid's auc: 0.834164	valid's binary_logloss: 0.248907
[75]	build's auc: 0.847329	build's binary_logloss: 0.238368	valid's auc: 0.834414	valid's binary_logloss: 0.248696
[80]	build's auc: 0.847725	build's binary_logloss: 0.238019	valid's auc: 0.83464	valid's binary_logloss: 0.248501
[85]	build's auc: 0.848165	build's binary_logloss: 0.237698	valid's auc: 0.834797	valid's binary_logloss: 0.248371
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[90]	bu

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[430]	build's auc: 0.857811	build's binary_logloss: 0.231787	valid's auc: 0.836091	valid's binary_logloss: 0.247529
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[435]	build's auc: 0.857853	build's binary_logloss: 0.231763	valid's auc: 0.836087	valid's binary_logloss: 0.247531
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warnin

[125]	build's auc: 0.850595	build's binary_logloss: 0.23637	valid's auc: 0.840463	valid's binary_logloss: 0.240906
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[130]	build's auc: 0.850784	build's binary_logloss: 0.236254	valid's auc: 0.840513	valid's binary_logloss: 0.240875
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[135]	build's auc: 0.851008	build's binary_logloss: 0.236114	valid's auc: 0.840548	valid's binary_logloss: 0.240854
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[140]	build's auc: 0.851186	build's binary_logloss: 0

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[250]	build's auc: 0.854195	build's binary_logloss: 0.234131	valid's auc: 0.840863	valid's binary_logloss: 0.240631
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[255]	build's auc: 0.854328	build's binary_logloss: 0.234052	valid's auc: 0.840871	valid's binary_logloss: 0.240627
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[260]	build's auc: 0.854402	build's binary_logloss: 0.234008	valid's auc: 0.840861	valid's bi

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[360]	build's auc: 0.856331	build's binary_logloss: 0.23288	valid's auc: 0.840913	valid's binary_logloss: 0.240584
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[365]	build's auc: 0.856495	build's binary_logloss: 0.232784	valid's auc: 0.840935	valid's binary_logloss: 0.24057
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[370]	build's auc: 0.856533	build's binary_logloss: 0.232763	valid's auc: 0.840924	valid's binary_logloss: 0.240575
[LightGBM] [Warning] No further splits with positive g

[465]	build's auc: 0.858184	build's binary_logloss: 0.231795	valid's auc: 0.840916	valid's binary_logloss: 0.240578
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Early stopping, best iteration is:
[409]	build's auc: 0.857312	build's binary_logloss: 0.232297	valid's auc: 0.840944	valid's binary_logloss: 0.240559


In [8]:
cscf_3.fit(x_train, labels)
stack1 = cscf_3.predict(x_test, test)
stack1.to_csv("data/lgb_stack3.csv", index=False)

build user:124611, valid user:6598
FINAL SHAPE
dbuild.shape:(8048078, 78)  dvalid.shape:(426583, 78)

[LightGBM] [Info] Number of positive: 787676, number of negative: 7260402
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.368235 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 28108
[LightGBM] [Info] Number of data points in the train set: 8048078, number of used features: 78
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.097871 -> initscore=-2.221104
[LightGBM] [Info] Start training from score -2.221104
Training until validation scores don't improve for 60 rounds
[5]	build's auc: 0.83199	build's binary_logloss: 0.28841	valid's auc: 0.8311	valid's binary_logloss: 0.286016
[10]	build's auc: 0.833687	build's binary_logloss: 0.272682	valid's auc: 0.831861	valid's binary_logloss: 0.270738
[15]	build's auc: 0.835032	build's binary_log

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[185]	build's auc: 0.852882	build's binary_logloss: 0.235002	valid's auc: 0.839512	valid's binary_logloss: 0.239991
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[190]	build's auc: 0.853006	build's binary_logloss: 0.234929	valid's auc: 0.839527	valid's binary_logloss: 0.23998
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[195]	build's auc: 0.853164	build's binary_logloss: 0.23482	valid's auc: 0.839538	valid's binary_logloss: 0.239965
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[300]	build's auc: 0.855326	build's binary_logloss: 0.233525	valid's auc: 0.83973	valid's binary_logloss: 0.23985
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[305]	build's auc: 0.855447	build's binary_logloss: 0.233452	valid's auc: 0.839732	valid's binary_logloss: 0.239845
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[310]	build's auc: 0.855564	build's binary_logloss: 0.23338	valid's auc: 0.839726	valid's binary_logloss: 0.23985
[LightGBM] [Warning] No further splits with positive gai

[410]	build's auc: 0.857443	build's binary_logloss: 0.232263	valid's auc: 0.839757	valid's binary_logloss: 0.239812
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[415]	build's auc: 0.857523	build's binary_logloss: 0.232218	valid's auc: 0.83974	valid's binary_logloss: 0.239819
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[420]	build's auc: 0.857624	build's binary_logloss: 0.232159	valid's auc: 0.839736	valid's binary_logloss: 0.23982
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[125]	build's auc: 0.850754	build's binary_logloss: 0.236212	valid's auc: 0.837351	valid's binary_logloss: 0.243511
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[130]	build's auc: 0.850933	build's binary_logloss: 0.236098	valid's auc: 0.837391	valid's binary_logloss: 0.24349
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[135]	build's auc: 0.851163	build's binary_logloss: 0.235948	valid's auc: 0.83748	valid's binary_logloss: 0.243442
[LightGBM] [Warning] No further splits with positive g

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[245]	build's auc: 0.854261	build's binary_logloss: 0.234054	valid's auc: 0.837787	valid's binary_logloss: 0.243239
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[250]	build's auc: 0.854369	build's binary_logloss: 0.233984	valid's auc: 0.837813	valid's binary_logloss: 0.243224
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[255]	build's auc: 0.854498	build's binary_logloss: 0.233907	valid's auc: 0.837831	valid's binary_logloss: 0.243213
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive

[55]	build's auc: 0.844401	build's binary_logloss: 0.241022	valid's auc: 0.837211	valid's binary_logloss: 0.242337
[60]	build's auc: 0.845194	build's binary_logloss: 0.240319	valid's auc: 0.837494	valid's binary_logloss: 0.241949
[65]	build's auc: 0.845924	build's binary_logloss: 0.23971	valid's auc: 0.837755	valid's binary_logloss: 0.241629
[70]	build's auc: 0.846719	build's binary_logloss: 0.239122	valid's auc: 0.837983	valid's binary_logloss: 0.241373
[75]	build's auc: 0.847142	build's binary_logloss: 0.238738	valid's auc: 0.838256	valid's binary_logloss: 0.241136
[80]	build's auc: 0.847569	build's binary_logloss: 0.238369	valid's auc: 0.838509	valid's binary_logloss: 0.240927
[85]	build's auc: 0.848014	build's binary_logloss: 0.238043	valid's auc: 0.838681	valid's binary_logloss: 0.240791
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[90]	build's auc: 0.848346	build's binary_logloss: 0.237789	valid's auc: 0.83882	valid's binary_logloss: 0.240686
[LightG

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[210]	build's auc: 0.853287	build's binary_logloss: 0.234724	valid's auc: 0.839781	valid's binary_logloss: 0.240052
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[215]	build's auc: 0.853405	build's binary_logloss: 0.234653	valid's auc: 0.839801	valid's binary_logloss: 0.240041
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[220]	build's auc:

[320]	build's auc: 0.855519	build's binary_logloss: 0.23338	valid's auc: 0.839963	valid's binary_logloss: 0.239938
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[325]	build's auc: 0.855572	build's binary_logloss: 0.233349	valid's auc: 0.83995	valid's binary_logloss: 0.239945
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[330]	build's auc: 0.855672	build's binary_logloss: 0.233292	valid's auc: 0.839944	valid's binary_logloss: 0.239948
[LightGBM] [Warning] No further splits with positive g

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[435]	build's auc: 0.857584	build's binary_logloss: 0.232148	valid's auc: 0.840034	valid's binary_logloss: 0.239901
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[440]	build's auc: 0.857658	build's binary_logloss: 0.232104	valid's auc: 0.840035	valid's binary_logloss: 0.239901
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warnin